<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Luis Figueroa
- Nombre de alumno 2: Felipe Méndez


### **Link de repositorio de GitHub:** `https://github.com/Felipe-mendezp/Labs-Figueroa-Mendez`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import os
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Laboratorios'
except: 
    print('Ignorando conexión drive-colab')

os.chdir(path)

Mounted at /content/drive


In [ ]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [ ]:
# Creamos la variable Label (1 si es seleccionado, 0 si no)
df_players['Label'] = df_players['National_Position'].apply(lambda x: 1 if not pd.isnull(x) else 0)

# Vemos como esta distribuida la clase
print(df_players['Label'].value_counts())

0    16513
1     1075
Name: Label, dtype: int64


Vemos que existe un desbalance de clases, pues existen 16 veces más datos de jugadores que no son seleccionados de los que si lo son. Esto podría tener implicancias a la hora de entrenar modelos.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [ ]:
# Definir las transformaciones para cada columna
column_transformer = ColumnTransformer([
    ('numeric', StandardScaler(), ['Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 
                                   'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 
                                   'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 
                                   'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 
                                   'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 
                                   'Penalties', 'Volleys']),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), ['Nationality', 'Preffered_Foot']),
    ('minmax', MinMaxScaler(), ['Height', 'Weight'])
])

# Aplicar las transformaciones al conjunto de datos
preprocessed_data = column_transformer.fit_transform(df_players)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Crear los pipelines con diferentes clasificadores
pipeline_rf = Pipeline([('preprocessing', column_transformer), ('classifier', RandomForestClassifier())])

pipeline_lr = Pipeline([('preprocessing', column_transformer), ('classifier', LogisticRegression())])

pipeline_svc = Pipeline([('preprocessing', column_transformer), ('classifier', SVC())])

En el ColumnTransformer, se aplican transformaciones específicas a diferentes tipos de variables de acorde a lo observado en el  `Player_Stats_Report.html`. Para las variables numéricas, se utiliza el StandardScaler, que estandariza las características al centrarlas alrededor de cero y escalarlas para tener una varianza unitaria. Esto ayuda a evitar que una característica tenga más influencia que otras durante el entrenamiento del modelo. Para las variables categóricas, se utiliza el OneHotEncoder, que codifica cada categoría en una columna binaria separada, capturando así la información de las diferentes categorías sin establecer un orden numérico. Además, se utiliza el MinMaxScaler para las variables de escala mínima y máxima, lo que ajusta los valores dentro de un rango específico (por defecto, entre 0 y 1) preservando las relaciones proporcionales entre las características. Estas transformaciones se eligen para adecuar los datos al modelo predictivo y garantizar que todas las variables estén en un formato adecuado para el análisis.






Los modelos seleccionados en base a la guía para los pipelines son el RandomForestClassifier, LogisticRegression y SVC. El RandomForestClassifier es elegido por su capacidad para manejar diferentes tipos de variables y capturar relaciones no lineales y complejas en los datos. La LogisticRegression es seleccionada por su interpretabilidad y capacidad para estimar probabilidades de pertenencia a una clase específica. El SVC es elegido debido a su capacidad para separar eficientemente conjuntos de datos en espacios de alta dimensión y detectar patrones no lineales. Estas elecciones se basan en la idoneidad de los modelos para la clasificación binaria y su capacidad para manejar las características específicas del conjunto de datos.

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [ ]:
# Entrenamos el modelo sin la variable National_Position pues es lo que queremos predecir
df_filter = df_players.drop('National_Position', axis=1)

# Dividimos los datos en conjunto de train y test
X_train, X_test, y_train, y_test = train_test_split(df_filter.drop('Label', axis=1), df_filter['Label'], 
                                                    test_size=0.2, stratify=df_filter['Label'], random_state=1234)

In [ ]:
# Entrenamos los pipelines
pipeline_rf.fit(X_train, y_train)
pipeline_lr.fit(X_train, y_train)
pipeline_svc.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  ['Age', 'Weak_foot',
                                                   'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Nationality',
                                                   'Preffered_Foot']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['Height', 'Weight'])])),
                ('classifier', SVC())])

In [ ]:
def evaluate_model(model,X_train,y_train,X_test,y_test):
  y_test_pred = model.predict(X_test)
  y_train_pred = model.predict(X_train)

  train_accuracy = round(accuracy_score(y_train, y_train_pred),4)
  train_f1 = round(f1_score(y_train, y_train_pred, average='weighted'),4)
  test_accuracy = round(accuracy_score(y_test, y_test_pred),4)
  test_f1 = round(f1_score(y_test, y_test_pred, average='weighted'),4)

  print("Métricas de evaluación - Conjunto de entrenamiento:")
  print("Accuracy:", train_accuracy)
  print("F1-score:", train_f1)

  print("\nMétricas de evaluación - Conjunto de prueba:")
  print("Accuracy:", test_accuracy)
  print("F1-score:", test_f1)

  print(classification_report(y_test, y_test_pred))

In [ ]:
# Hacemos las predicciones en el conjunto de test
y_pred_rf = pipeline_rf.predict(X_test)
y_pred_lr = pipeline_lr.predict(X_test)
y_pred_svc = pipeline_svc.predict(X_test)

# Métricas de evaluación
print("Random Forest Classifier:")
print(classification_report(y_test, y_pred_rf))

print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))

print("Support Vector Classifier:")
print(classification_report(y_test, y_pred_svc))

Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.74      0.08      0.14       215

    accuracy                           0.94      3518
   macro avg       0.84      0.54      0.56      3518
weighted avg       0.93      0.94      0.92      3518

Logistic Regression:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      3303
           1       0.68      0.27      0.39       215

    accuracy                           0.95      3518
   macro avg       0.82      0.63      0.68      3518
weighted avg       0.94      0.95      0.94      3518

Support Vector Classifier:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.67      0.01      0.02       215

    accuracy                           0.94      3518
   macro avg       0.80      0.50      0.49      3518


**Respuesta**:
El accurayc es una medida de que tan precisa es al prediccion del modelo, en el caso de la clase positiva, nos dice el porcentaje de predicciones corerctas.
La precisión es una medida de qué tan preciso es el modelo en la clasificación de la clase positiva. Se calcula dividiendo el número de verdaderos positivos por la suma de verdaderos positivos y falsos positivos. El F1-score es una medidca de qué tan bien el modelo puede encontrar todos los casos positivos. Se calcula dividiendo el número de verdaderos positivos por la suma de verdaderos positivos y falsos negativos.


Se utilizaron 3 modelos siguiendo la guia mencionada, los cuales obtuvieron buenos resultados en general, sin embargo, estos resultados estan desbalanceados, es decir, los resultados en la clase negativa fueron mucho mejores que en la clase positiva. Lo anterior, es resultado del desbalance que hay en la variable a predecir. Por lo tanto, para mejorar los resultados se propone conseguir mas datos y balancear las clases.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [ ]:
# Llevamos a string la variable Club_Position
df_players['Club_Position'] = df_players['Club_Position'].astype(str)

# Diccionario con las posiciones
posiciones = {
    'ataque': ['ST', 'CF'],
    'central_ataque': ['RW', 'CAM', 'LW'],
    'central': ['RM', 'CM', 'LM'],
    'central_defensa': ['RWB', 'CDM', 'LWB'],
    'defensa': ['RB', 'CB', 'LB'],
    'arquero': ['GK']
    }

# Creamos nueva columna con las posiciones
df_players['Position'] = df_players['Club_Position'].map(lambda x: next((k for k, v in posiciones.items() if x in v), None))

In [ ]:
# Print de número de filas por clase
print(df_players['Position'].value_counts())

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Position, dtype: int64


In [ ]:
df_filter_position = df_players.dropna(subset=['Position'])

In [ ]:

df_filter_position = df_filter_position.drop(['National_Position','Club_Position','Label'], axis=1)

encoder = LabelEncoder()
df_filter_position['Position'] = encoder.fit_transform(df_filter_position['Position'])

label_mappings = {index: label for index, label in enumerate(encoder.classes_)}
for encoded_label, original_label in label_mappings.items():
    print(f"Etiqueta codificada {encoded_label} corresponde a '{original_label}'")

df_filter_position.head()

Etiqueta codificada 0 corresponde a 'arquero'
Etiqueta codificada 1 corresponde a 'ataque'
Etiqueta codificada 2 corresponde a 'central'
Etiqueta codificada 3 corresponde a 'central_ataque'
Etiqueta codificada 4 corresponde a 'central_defensa'
Etiqueta codificada 5 corresponde a 'defensa'


,Name,Nationality,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Position
0,Cristiano Ronaldo,Portugal,185,80,Right,32,High / Low,4,5,93,...,95,85,92,93,90,81,76,85,88,3
1,Lionel Messi,Argentina,170,72,Left,29,Medium / Medium,4,4,95,...,68,71,85,95,88,89,90,74,85,3
2,Neymar,Brazil,174,68,Right,25,High / Medium,5,5,95,...,61,62,78,89,77,79,84,81,83,3
3,Luis Suárez,Uruguay,182,85,Right,30,High / Medium,4,4,91,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,193,92,Right,31,Medium / Medium,4,1,48,...,78,25,25,13,16,14,11,47,11,0


In [ ]:
column_transformer_pos = ColumnTransformer([
    ('numeric', StandardScaler(), ['Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 
                                   'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 
                                   'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 
                                   'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 
                                   'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 
                                   'Penalties', 'Volleys']),
    ('categorical', OneHotEncoder(handle_unknown='ignore'), ['Nationality', 'Preffered_Foot','Work_Rate']),
    ('minmax', MinMaxScaler(), ['Height', 'Weight'])
])



pipeline_pos = Pipeline([('preprocessing', column_transformer_pos), ('classifier', SVC())])


In [ ]:
X = df_filter_position.drop(['Name','Position'],axis=1)
y = df_filter_position['Position']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir la forma de los conjuntos de entrenamiento y prueba
print("Forma del conjunto de entrenamiento:", X_train.shape)
print("Forma del conjunto de prueba:", X_test.shape)

Forma del conjunto de entrenamiento: (3151, 36)
Forma del conjunto de prueba: (788, 36)


In [ ]:
pipeline_pos.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  ['Age', 'Weak_foot',
                                                   'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Nationality',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['Height', 'Weight'])])),
                ('classifier', SVC())])

In [ ]:

# Métricas de evaluación
print("Support Vector Classifier:\n")
evaluate_model(pipeline_pos,X_train,y_train,X_test,y_test)



Support Vector Classifier:

Métricas de evaluación - Conjunto de entrenamiento:
Accuracy: 0.7991
F1-score: 0.7794

Métricas de evaluación - Conjunto de prueba:
Accuracy: 0.7538
F1-score: 0.7305
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       125
           1       0.80      0.84      0.82        86
           2       0.60      0.70      0.65       192
           3       0.55      0.29      0.38       114
           4       0.40      0.15      0.21        41
           5       0.82      0.97      0.89       230

    accuracy                           0.75       788
   macro avg       0.69      0.66      0.66       788
weighted avg       0.73      0.75      0.73       788



**Respuesta**:

Se decidio utilizar un modelo de clasificacion de Support Vector Machines, siguiendo la guia indicada. 

Al analizar los resultados, se puede observar que el modelo obtuvo un buen accuracy y f1-score, los cuales tienen valores similares en el conjunto de entrenamiento y prueba. Se puede concluir entonces, que no estamos frente a un gran problema de overfitting, sin embargo, estos resultados pueden mejorarse de dos maneras. En primer lugar, se debe conseguir mas datos, debido a que el filtrado por posiciones significo que no se puedan utilizar todos los datos. La segunda manera de mejorar los modelos consiste en balancear las clases a predecir, pues estas estan muy desbalanceadas. Para lograr lo anterior, se debiera utilizar tecnicas de samplig, como random over sampling, SMOTE, entre otros.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [X] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [X] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [ ]:
df_salarios = pd.read_csv('salarios.csv').drop('Unnamed: 0',axis=1)
df_salarios

,Player,Weekly Salary
0,Lionel Messi,1538905.0
1,Cristiano Ronaldo,1248536.0
2,Antoine Griezmann,997032.0
3,Neymar,797726.0
4,Gareth Bale,607706.0
...,...,...
2529,Simone Ghidotti,283.0
2530,Valentin Gendrey,100.0
2531,John Yeboah,70.0
2532,Antonis Aidonis,70.0


In [ ]:
data = df_players.merge(df_salarios, left_on='Name', right_on='Player', how='left').drop(['Player','Position','National_Position'],axis=1).dropna().reset_index(drop=True)
print(data.shape)
data

(1866, 40)


,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Label,Weekly Salary
0,Cristiano Ronaldo,Portugal,LW,185,80,Right,32,High / Low,4,5,...,85,92,93,90,81,76,85,88,1,1248536.0
1,Lionel Messi,Argentina,RW,170,72,Left,29,Medium / Medium,4,4,...,71,85,95,88,89,90,74,85,1,1538905.0
2,Neymar,Brazil,LW,174,68,Right,25,High / Medium,5,5,...,62,78,89,77,79,84,81,83,1,797726.0
3,Luis Suárez,Uruguay,ST,182,85,Right,30,High / Medium,4,4,...,77,87,94,86,86,84,85,88,1,508923.0
4,Manuel Neuer,Germany,GK,193,92,Right,31,Medium / Medium,4,1,...,25,25,13,16,14,11,47,11,1,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Phillip Menzel,Germany,Res,191,83,Right,18,Medium / Medium,3,1,...,10,25,5,10,12,8,15,5,0,2034.0
1862,Manuel Akanji,Switzerland,Sub,187,85,Right,21,Medium / Medium,2,2,...,51,38,25,18,26,27,40,31,0,54176.0
1863,Moritz Nicolas,Germany,Res,195,87,Right,19,Medium / Medium,2,1,...,16,22,9,10,17,14,17,11,0,2262.0
1864,Giacomo Satalino,Italy,Sub,188,70,Right,17,Medium / Medium,1,1,...,11,19,6,5,13,11,18,9,0,2827.0


In [ ]:
X = data.drop(['Name','Weekly Salary'],axis=1)
y = data['Weekly Salary']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir la forma de los conjuntos de entrenamiento y prueba
print("Forma del conjunto de entrenamiento:", X_train.shape)
print("Forma del conjunto de prueba:", X_test.shape)

Forma del conjunto de entrenamiento: (1492, 38)
Forma del conjunto de prueba: (374, 38)


In [ ]:
pipeline = Pipeline([('preprocessing', column_transformer_pos), ('classifier',RandomForestRegressor())])


In [ ]:
pipeline.fit(X_train,y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  ['Age', 'Weak_foot',
                                                   'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Aggression', 'Reactions',
                                                   'Interceptions', 'Vision',
                                                   'Composure', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Acceleration', 'Speed',
                                                   'Stamina', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', ...]),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Nationality',
                                                   'Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['Height', 'Weight'])])),
                ('classifier', RandomForestRegressor())])

In [ ]:
def evaluate_regression_model(model, X_train, y_train, X_test, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    train_mape = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100
    test_mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100

    print("Métricas de evaluación - Conjunto de entrenamiento:")
    print("MSE:", train_mse)
    print("R²:", train_r2)
    print("MAPE:", train_mape)

    print("\nMétricas de evaluación - Conjunto de prueba:")
    print("MSE:", test_mse)
    print("R²:", test_r2)
    print("MAPE:", test_mape)

In [ ]:
# Métricas de evaluación
print("Random Forest Regression:\n")

evaluate_regression_model(pipeline,X_train,y_train,X_test,y_test)


Random Forest Regression:

Métricas de evaluación - Conjunto de entrenamiento:
MSE: 377384295.5539531
R²: 0.9501439074710006
MAPE: 37.967595731935894

Métricas de evaluación - Conjunto de prueba:
MSE: 2001747928.125551
R²: 0.3878828914522301
MAPE: 92.61422493882337


**Respuesta**:


El $R^2$ es una métrica utilizada en problemas de regresión para evaluar qué proporción de la variabilidad de la variable dependiente puede ser explicada por el modelo de regresión. $R^2$ varía entre 0 y 1. Un valor de  $R^2$ cercano a 1 indica que el modelo es capaz de explicar una gran parte de la variabilidad de la variable dependiente, mientras que un valor cercano a 0 indica que el modelo no puede explicar bien la variabilidad observada.


Al momento de realizar la union de las base de datos, solo se pudo obtener el salario de 1866 jugadores, es decir, solo se podran utilizar estos datos al momento de entrenar los modelos para realizar la prediccion. Al tener pocos datos, siguiendo la guia mencionada, se decidio utilizar un modelo de ensamblaje de Random Forest, debido a que los modelos de regresion lineal tenian muy malos resultados.

Al analizar los resultados, vemos que el modelo de regresion de random forest obtuvo un  $R^2$ mayor a 0.35 en el conjunto de prueba. En este contexto, la razon los malos resultados en general, se debe principalemte a la poca cantidad de datos, puesto que se cuenta en principio con una gran cantidad de features.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>